## tCoIR - Text Analysis
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [2]:
%load_ext autoreload
%autoreload 2

from beakerx.object import beakerx
from beakerx import *

from IPython.display import display #, set_matplotlib_formats
import text_analytic_tools.utility as utility

utility.setup_default_pd_display(pd)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


steps: Raw text corpus => Prepped text corpus => textaCy corpus => filtered & tokenized corpus => word co-occurrence

Load (or create a new) textaCy corpus from the source text files.

In [ ]:

container = textacy_utility.load_or_create(
    source_path='/home/roger/source/text_analytic_tools/data/tCoIR/tCoIR_en_45-72.txt.zip',
    language='en',
    document_index=None,
    merge_entities=False,
    overwrite=False,
    use_compression=True,
    disabled_pipes=tuple(("ner", "parser", "textcat"))
)

corpus = current_corpus_container().textacy_corpus
corpus_path = current_corpus_container().prepped_source_path


Prepare a filtered tokenized corpus.

In [ ]:

docs = ((doc._.meta['filename'], doc) for doc in corpus)
tokenized_docs = textacy_utility.extract_document_tokens(docs, **opts)
pos_tags = domain_logic.pos_tags()
document_index = domain_logic.document_index(corpus)

filepath, df_summary = common.store_tokenized_corpus_as_archive(
    tokenized_docs,
    corpus_source_filepath,
    term_substitutions=term_substitutions(corpus),
    min_freq=gui.min_freq.value,
    max_doc_freq=gui.max_doc_freq.value,
    substitute_terms=gui.substitute_terms.value,
    ngrams=gui.ngrams.value,
    min_word=gui.min_word.value,
    normalize=gui.normalize.value,
    filter_stops=gui.filter_stops.value,
    filter_punct=gui.filter_punct.value,
    named_entities=gui.named_entities.value,
    include_pos=gui.include_pos.value,
    chunk_size=gui.chunk_size.value,
    word_counts=word_counts,
    word_document_counts=word_document_counts
)

logger.info("Done! Result stored in '{}'".format(filepath))

display(df_summary)

In [13]:
import time
import text_analytic_tools.text_analysis.co_occurrence as co_occurrence
import text_analytic_tools.common.text_corpus as text_corpus

from text_analytic_tools.domain_logic_config import current_domain as domain_logic

def compute_co_occurrence(filepath, window_size=5, distance_metric=0, method='HAL', normalize='size'):

    corpus = text_corpus.SimplePreparedTextCorpus(filepath, lowercase=True)
    document_index = domain_logic.compile_documents(corpus)

    df = co_occurrence.compute(corpus, document_index, window_size, distance_metric, normalize, method)

    result_filename = '{}_{}_result_co_occurrence_{}.xlsx'.format(method, window_size, time.strftime("%Y%m%d_%H%M"))
    df.to_excel(result_filename)
    print('Result saved to file {}'.format(result_filename))
    print('Now you are ready to do some serious stuff!')


https://github.com/maciejkula/glove-python/issues/96

```bash
% git clone https://github.com/maciejkula/glove-python.git
% cd glove-python/
% cd glove/
% cython glove_cython.pyx
% cythonize glove_cython.pyx
% cython metrics/accuracy_cython.pyx
% cythonize metrics/accuracy_cython.pyx
% cython --cplus corpus_cython.pyx
% cythonize corpus_cython.pyx
% cd ..
% python setup.py cythonize
% make
% pip install -e .
```

  